In [36]:
import pandas as pd 
from sklearn.cross_validation import train_test_split
import Recommenders as Recommenders

ModuleNotFoundError: No module named 'Recommenders'

In [12]:
# File paths
triplets_file = r"D:\ONE DRIVE MAHI\OneDrive\Documents\GitHub\Music-Recommender-KNN\song_dataset\triplets_file.csv"
songs_metadata_file = r"D:\ONE DRIVE MAHI\OneDrive\Documents\GitHub\Music-Recommender-KNN\song_dataset\song_data.csv"

# Load the triplets file (with header in the first row)
song_df_1 = pd.read_csv(triplets_file)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']

# Load the songs metadata file
song_df_2 = pd.read_csv(songs_metadata_file)

song_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left")

print(song_df.head())


                                    user_id             song_id  listen_count  \
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995             1   
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B             2   
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBXHDL12A81C204C0             1   
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBYHAJ12A6701BF1D             1   
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SODACBL12A8C13C273             1   

             title                        release    artist_name  year  
0         The Cove             Thicker Than Water   Jack Johnson     0  
1  Entre Dos Aguas            Flamenco Para Niños  Paco De Lucia  1976  
2         Stronger                     Graduation     Kanye West  2007  
3   Constellations              In Between Dreams   Jack Johnson  2005  
4     Learn To Fly  There Is Nothing Left To Lose   Foo Fighters  1999  


In [23]:
#Checking listening percentage
#We are checking how many users interacted with the song, not how many times they've individually listened to it.
song_grouped=song_df.groupby(['title']).agg({'listen_count':'count'}).reset_index()
#Here listen count now represents how many users have listened to that song
grouped_sum=song_grouped['listen_count'].sum() #Summing up
song_grouped['percentage']=song_grouped['listen_count'].div(grouped_sum)*100
# Sort the DataFrame temporarily without modifying the original
sorted_song_grouped = song_grouped.sort_values(['listen_count', 'title'], ascending=[False, True])

# View the sorted result
print(sorted_song_grouped.head())


                               title  listen_count  percentage
6836                   Sehr kosmisch          8277     0.41385
8725                            Undo          7032     0.35160
1964  Dog Days Are Over (Radio Edit)          6949     0.34745
9496                  You're The One          6729     0.33645
6498                         Revelry          6145     0.30725


In [ ]:
#Return unique users and unique songs
users = song_df['user_id'].unique()
len(users)


76353

In [ ]:
songs = song_df['title'].unique()
len(songs) 

9567

In [33]:
train_data, test_data = train_test_split(song_df, test_size = 0.20, random_state=0)

In [34]:
#Define recommeneders module
class popularity_recommender_py():
    def __init__(self):
        self.train_data=None
        self.user_id=None
        self.item_id=None
        self.popularity_recommendations=None
    
    def create(self, train_data, user_id, item_id):
        self.train_data=train_data
        self.user_id=user_id
        self.item_id=item_id
        #Count of user_ids for each unique song as rec score
        train_data_grouped=train_data.groupby([self.item_id]).agg({self.user_id:'count'}).reset_index()
        #Renaming Column user_id to score
        train_data_grouped.rename(columns = {'user_id': 'score'}, inplace=True)
        train_data_sort=train_data_grouped.sort_values(['score', self.item_id], ascending=[0, 1])
        train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
        self.popularity_recommendations=train_data_sort.head(10) #Gets the top ten recommendations

    def recommend(self, user_id):
        user_recommendations = self.popularity_recommendations
        user_recommendations['user_id'] = user_id
        #Reordering columns for convenience
        cols = user_recommendations.columns.tolist()        
        cols = cols[-1:] + cols[:-1]        
        user_recommendations = user_recommendations[cols]
        return user_recommendations



In [38]:
pm = popularity_recommender_py()
pm.create(train_data, 'user_id', 'title')
#user the popularity model to make some prediction
user_id = users[5]
pm.recommend(user_id)

,user_id,title,score,Rank
6836,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Sehr kosmisch,6630,1.0
8725,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Undo,5639,2.0
1964,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Dog Days Are Over (Radio Edit),5592,3.0
9496,4bd88bfb25263a75bbdd467e74018f4ae570e5df,You're The One,5396,4.0
6498,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Revelry,4938,5.0
6825,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Secrets,4627,6.0
3437,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Horn Concerto No. 4 in E flat K495: II. Romanc...,4368,7.0
2595,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Fireflies,3835,8.0
3322,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Hey_ Soul Sister,3819,9.0
8494,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Tive Sim,3707,10.0
